# Case Satander

Ajude o **Santander** a identificar o valor das transações para cada cliente potencial. Esse é um primeiro passo que o Santander precisa acertar para personalizar seus serviços em grande escala.

Base de dados: https://www.kaggle.com/c/santander-value-prediction-challenge/data

O case podera ser quebrado nas **6** partes seguintes:  
- **Identificar o problema**
  - Qual o tipo (classificaçao, regressao, clustering) de problema e porque? 
- **Necessidades de aplicar transformações?**
  - Ex: *imputing* de valores null, *encoding* de colunas *string*, etc 
- **Separar os sets de treinamento e teste**
- **Baseline**
  - Achar uma baseline, um primeiro modelo para ter uma referência
- **Escolher a métrica**
- **Melhorar o resultado**
  - Feature engineering, otimização do modelo, hiperparametros, etc

## Importar Bibliotecas e funções

In [89]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import statsmodels.api as sm
from sklearn import metrics


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [90]:
def predict_table(y, y_pred, columns=['Real', 'Previsto'], residual=False):
    """Tabela de previsão e cálculo de resíduo do modelo

    Args:
        y (pd.DataFrame): variável resposta
        fit (.predict): predição do modelo ajustado dos regressores
        columns (list): nome das colunas Real e Previsto
        residual (bool, optional): cálculo do residual dos regressores. Defaults to False.

    Returns:
        pred(pd.DataFrame): base com predição do modelo ajustado
    """  
    
    import pandas as pd
    
    pred = pd.concat([y, pd.DataFrame(y_pred)], axis=1)
    pred.columns = columns

    if residual is True:
        pred['Res'] = pred[columns[0]] - pred[columns[1]]

    return pred

In [91]:
def adj_quality(pred, index, form=False, reg=None):
    """Qualidade de ajuste do modelo com R² e RMSE

    Args:
        pred (pd.DataFrame): tabela de previsão do modelo

    Returns:
        adj(pd.DataFrame): cálculo de R² e RMSE do modelo
    """    
    import pandas as pd
    import numpy as np
    from sklearn import metrics
    
    pred.columns = ['Real', 'Previsto']

    
    if form is True:
        fo = "y = {} + {} x".format(reg.intercept_, reg.coef_[0])
        adj = pd.DataFrame({
        'R²': metrics.r2_score(pred.loc[:, pred.columns[0]], 
                               pred.loc[:, pred.columns[1]]),
        'MSE': metrics.mean_squared_error(pred.loc[:, pred.columns[0]], 
                                          pred.loc[:, pred.columns[1]]),
        'RMSE': np.sqrt(metrics.mean_squared_error(pred.loc[:, pred.columns[0]], 
                                                   pred.loc[:, pred.columns[1]]))
        }, index=[index])
        
        return adj.round(decimals=3), fo
    
    else:
        adj = pd.DataFrame({
            'R²': metrics.r2_score(pred['Real'], 
                                   pred['Previsto']),
            'MSE': metrics.mean_squared_error(pred['Real'], 
                                              pred['Previsto']),
            'RMSE': metrics.mean_squared_error(pred['Real'], 
                                               pred['Previsto'], squared=False),
            'RMSLE': metrics.mean_squared_log_error(pred.loc[pred['Previsto'] > 0, ['Real']], 
                                                    pred.loc[pred['Previsto'] > 0, ['Previsto']]),
            'MAE': metrics.mean_absolute_error(pred['Real'], 
                                               pred['Previsto'])
        }, index=[index])
        
        return adj.round(decimals=3)

## Importação e entendimento dos dados

In [92]:
df = pd.read_csv("C:/Users/bruna/OneDrive/Documentos/Estudos/Tech/DNC/#7 Modelos Supervisionados/train.csv/train.csv")

In [93]:
df.shape

(4459, 4993)

In [94]:
df.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB


In [96]:
df['ID'].describe()

count          4459
unique         4459
top       000d6aaf2
freq              1
Name: ID, dtype: object

In [97]:
df['target'].describe()

count    4.459000e+03
mean     5.944923e+06
std      8.234312e+06
min      3.000000e+04
25%      6.000000e+05
50%      2.260000e+06
75%      8.000000e+06
max      4.000000e+07
Name: target, dtype: float64

In [98]:
desc = df.describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99], include='all').T

A maioria das colunas é pouco representativa, mas os dados estão anonimizados e o modelo conseguirá extrapolar informações.

In [99]:
desc.loc[desc['75%'] > 0]

,count,unique,top,freq,mean,std,min,5%,25%,50%,75%,90%,95%,99%,max
target,4459.0,NaN,NaN,NaN,5944923.322036,8234311.50163,30000.0,70000.0,600000.0,2260000.0,8000000.0,20000000.0,24000000.0,38571428.58,40000000.0
20aa07010,4459.0,NaN,NaN,NaN,2569407.36837,9610183.073147,0.0,0.0,0.0,0.0,600000.0,6000000.0,15071200.0,43768000.0,319612000.0
963a49cdc,4459.0,NaN,NaN,NaN,2409617.084082,9125827.284724,0.0,0.0,0.0,0.0,423333.33,5200000.0,14149500.0,42000000.0,319612000.0
26fc93eb7,4459.0,NaN,NaN,NaN,2349408.591886,7815629.772086,0.0,0.0,0.0,0.0,500000.0,5354333.328,15567028.574,40350290.0028,115538000.0
0572565c2,4459.0,NaN,NaN,NaN,2584132.278116,9639046.695438,0.0,0.0,0.0,0.0,550000.0,5400000.0,17000000.0,49275000.0,319612000.0
66ace2992,4459.0,NaN,NaN,NaN,2496764.788334,9204054.372275,0.0,0.0,0.0,0.0,551666.67,5788800.0,15567028.574,41613333.3372,319612000.0
fb49e4212,4459.0,NaN,NaN,NaN,2432274.596932,8105497.214112,0.0,0.0,0.0,0.0,485000.0,5442100.0,15845714.278,42650000.0012,116000000.0
6619d81fc,4459.0,NaN,NaN,NaN,2552375.317919,9600044.115834,0.0,0.0,0.0,0.0,551000.0,5235620.508,16000000.0,47000000.0,319612000.0
6eef030c1,4459.0,NaN,NaN,NaN,2358790.950325,7641190.132478,0.0,0.0,0.0,0.0,526000.0,6000000.0,14669799.994,38764028.5764,115538000.0
fc99f9426,4459.0,NaN,NaN,NaN,2507030.588926,8428103.186808,0.0,0.0,0.0,0.0,535000.0,5400000.0,16721333.334,46806666.6628,100000000.0


In [100]:
desc.loc[desc['freq'] > 0]

,count,unique,top,freq,mean,std,min,5%,25%,50%,75%,90%,95%,99%,max
ID,4459,4459,000d6aaf2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Não existem valores nulos

In [101]:
desc.loc[desc['count'] < 4459]

,count,unique,top,freq,mean,std,min,5%,25%,50%,75%,90%,95%,99%,max


Como o objetivo é identificar valores numéricos contínuos, esse se trata de um problema de regressão. Existe apenas uma coluna classificada como String, sendo ela a de identificação dos clientes.

## Dividir a base de dados

In [102]:
id = df['ID']
x = df.drop(columns=['ID', 'target'])
y =  df['target']

In [103]:
x_Train, x_Test, y_Train, y_Test = train_test_split(x, y, test_size=0.2, random_state=42)

data = (x_Train, x_Test, y_Train, y_Test)

for d in data:
    d.reset_index(drop=True, inplace=True)

## Treinar uma baseline

In [104]:
reg = LinearRegression().fit(x_Train, y_Train)

Como temos mais colunas que linhas, o modelo terá uma forte tendência a presentar um overfitting

A métrica escolhida será o MAE pois ele representa o erro absoluto na mesma forma de y

In [105]:
y_pred_test = reg.predict(x_Test)

adj_quality(predict_table(y_Test, y_pred_test), index='Teste')

,R²,MSE,RMSE,RMSLE,MAE
Teste,-9.104167e+15,5.162928e+29,7.185352e+14,301.624,3.093645e+14


In [106]:
y_pred_train = reg.predict(x_Train)

adj_quality(predict_table(y_Train, y_pred_train), index='Treino')

,R²,MSE,RMSE,RMSLE,MAE
Treino,0.999,6.832091e+10,261382.696,0.014,13381.106


## Selecionar features

Como a base possui muitas features, vamos tentar selecionar 10%, avaliando os níveis de correlação entre as variáveis. Qual a  melhor quantidade de features para evitar um overfitting?

In [107]:
sel_kbest = SelectKBest(f_regression, k=45).fit(x_Train, y_Train)

c:\Users\bruna\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [108]:
x_Train_sel = sel_kbest.transform(x_Train)
x_Test_sel = sel_kbest.transform(x_Test)

In [109]:
x_Train.shape

(3567, 4991)

In [110]:
x_Train_sel.shape

(3567, 45)

## Escolher a métrica

In [114]:
reg = LinearRegression().fit(x_Train_sel, y_Train)

y_pred_test = reg.predict(x_Test_sel)

adj_quality(predict_table(y_Test, y_pred_test), index='Teste')

,R²,MSE,RMSE,RMSLE,MAE
Teste,0.099,5.109256e+13,7147905.818,3.745,5230263.099


In [116]:
y_pred_train = reg.predict(x_Train_sel)

adj_quality(predict_table(y_Train, y_pred_train), index='Treino')

,R²,MSE,RMSE,RMSLE,MAE
Treino,0.185,5.749941e+13,7582836.337,3.911,5322548.589


## Melhorar o resultado

Quanto mais árvores, menor a chance que ocorra um overfitting dos dados

In [118]:
reg = RandomForestRegressor(max_depth=6, random_state=0).fit(x_Train_sel, y_Train)

In [119]:
y_pred_test = reg.predict(x_Test_sel)

adj_quality(predict_table(y_Test, y_pred_test), index='Teste')

,R²,MSE,RMSE,RMSLE,MAE
Teste,0.228,4.375273e+13,6614584.803,3.72,4938382.638


In [121]:
y_pred_train = reg.predict(x_Train_sel)

adj_quality(predict_table(y_Train, y_pred_train), index='Treino')

,R²,MSE,RMSE,RMSLE,MAE
Treino,0.306,4.894332e+13,6995950.575,3.832,4946806.564


### Otimizar o modelo

In [123]:
parameters = {'max_depth': [2,6,10,20], 
              'n_estimators': [20,50,100,200]}

reg = RandomForestRegressor(random_state=0, n_jobs=-1)

clf = GridSearchCV(reg, parameters)
clf.fit(x_Train_sel, y_Train)

GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1, random_state=0),
             param_grid={'max_depth': [2, 6, 10, 20],
                         'n_estimators': [20, 50, 100, 200]})

In [124]:
clf.best_params_

{'max_depth': 20, 'n_estimators': 200}

In [126]:
y_pred_test = clf.predict(x_Test_sel)

adj_quality(predict_table(y_Test, y_pred_test), index='Teste')

,R²,MSE,RMSE,RMSLE,MAE
Teste,0.239,4.315576e+13,6569304.425,3.229,4707165.383


In [127]:
y_pred_train = clf.predict(x_Train_sel)

adj_quality(predict_table(y_Train, y_pred_train), index='Treino')

,R²,MSE,RMSE,RMSLE,MAE
Treino,0.628,2.622586e+13,5121118.95,2.642,3378944.532
